In [119]:
import os
import time
import calendar
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Label
from bokeh.charts import HeatMap
output_notebook()

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

Loading BokehJS ...

# Swedish Police Events Analysis

The purpose of this assignment is to collect data regarding events handled by the police, published on [Swedish police website](https://polisen.se/aktuellt/polisens-nyheter/) and analyze them.

The steps for achieving this are the following:

1. Build a web scraping algorithm in order to get data from website.
2. Clean and manipulate raw data in order to extract useful and usable information.
3. Analyze data in order to see how events cluster and how relate to each other.

## 1. Web Scraping

I build a function in order to collect data from Swedish police website. <br>

In general, it's easy to let the function running in the background as a cronjob and update the dataset with new events every t amount of time. However, since this assignment is a proof of concept, I will let it run for a fixed amount of time (namely 10 minutes) to collect data and then I will shut it off. I will also serialize the dataset, so I don't need to rerun it every time.

In [ ]:
def gather_data(seconds):
    """Gathers data of events from Swedish Police Website and returns them as a dataframe.
    
    Parameters
    ----------
    seconds : time
        Time to wait for gathering data (approximately 5-10 events per second)
    """
    
    # set options for headless chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1024x1400")

    # read chromedriver from current directory
    chrome_driver = os.path.join(os.getcwd(), "chromedriver")
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)

    # get url
    url = "https://polisen.se/aktuellt/polisens-nyheter/"
    driver.get(url)

    # start counter for loading events
    elapsed = 0
    start = time.time()
    time.perf_counter()
    
    while elapsed < seconds:
        # load more data by "clicking" loadMoreItems() javascript method on website
        elapsed = time.time() - start
        driver.find_element_by_css_selector(".more-items.police-element.center").click()
        element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.more-items.police-element.center'))
        WebDriverWait(driver, 20).until(element_present)

    # feed page source to bs4 to manipulate it
    bs = BeautifulSoup(driver.page_source)

    # read events from html
    events = bs.find_all("li", class_="list-item ng-scope contracted slide-animation-is-done")

    # read data from events and save them to lists
    date_list, subject_list, place_list, details_list = [], [], [], []

    for event in events:
        try:
            # read date, subject, place and details for events

            # clean flags from strings
            event_raw = event.strong.text.rstrip().lstrip()
            # split based on commas
            event_raw_list = event_raw.split(",")
            # check that everything is present (to avoid null values)
            if len(event_raw_list) == 3:
                # dates, subject, place
                date_list.append(event_raw_list[0])
                subject_list.append(event_raw_list[1])
                place_list.append(event_raw_list[2])

                # details
                details = event.find("span", class_ = "list-item-text").text.rstrip().lstrip()
                details_list.append(details)
        except:
            continue

    # save to dataframe
    dataframe = pd.DataFrame({
        "date":date_list,
        "place":place_list,
        "subject":subject_list,
        "details":details_list
    })
    
    # return dataframe
    return dataframe

In [ ]:
# collect data for analysis (uncomment to rerun it)
# raw_data = gather_data(600)

# serialize dataframe using pickle (uncomment to serialize after run gather_data())
# raw_data.to_pickle("./raw_data.pkl")

## 2. Manipulate Raw Data

Given the dataframe with raw data, I manipulate it in order to end up with a usable dataset for analysis and machine learning.

In [2]:
def manipulate_data(dataframe):
    """Receives a dataframe with raw data and manipulates them in order to become usable. It returns a dataframe.
    
    Parameters
    ----------
    dataframe : pandas dataframe
        dataframe with raw data to manipulate
    """
    # trim whitespace
    dataframe.place = dataframe.place.str.strip()
    dataframe.subject = dataframe.subject.str.strip()
    dataframe.details = dataframe.details.str.strip()
    
    # manipulate date
    # year (assume 2018 for eveything)
    dataframe['year'] = 2018
    
    # month (1-12 for Jan-Dec)
    dataframe['month'] = dataframe.date.apply(lambda row: row.split(" ")[1][0:3].capitalize()) 
    dataframe.month = dataframe.month.str.replace('Okt', 'Oct')
    cal = dict((v,k) for k,v in enumerate(calendar.month_abbr))
    dataframe = dataframe.replace({'month':cal})
    
    # day of week (1-7 for Mon-Sun)
    dataframe['day'] = dataframe.date.apply(lambda row: row.split(" ")[0])
    dataframe.day = pd.to_datetime(dataframe[['year', 'month', 'day']])
    dataframe.day = dataframe.day.dt.weekday_name
    
    # time (round to o'clock eg 6,7... 18:00 etc)
    dataframe['time'] = dataframe.date.apply(lambda row: row.split(" ")[2][0:2])
    
    # drop date
    dataframe = dataframe.drop("date", axis=1)
    
    # rename
    dataframe.rename(columns={"place":"city", "subject":"event"}, inplace=True)
    
    # rearrange
    dataframe = dataframe[["year", "month", "day", "time", "city", "event", "details"]]
    
    # drop summaries
    dataframe = dataframe[~dataframe.event.isin(["Sammanfattning natt", "Sammanfattning kväll och natt", "Sammanfattning förmiddag", 
                                                 "Sammanfattning kväll", "Sammanfattning vecka", "Sammanfattning eftermiddag"])]

    # group robberies
    for i in dataframe.index:
        if (dataframe.loc[i,'event'] =="Inbrott") | (dataframe.loc[i,'event'] =="Stöld/inbrott") | (dataframe.loc[i,'event'] =="Rån") | (dataframe.loc[i,'event'] == "Rån övrigt"):
            dataframe.loc[i,'event'] = "Stöld"
            
    # group cities
    cities = dataframe.city.value_counts()
    for city in cities[cities>=40].index:
        dataframe.loc[dataframe.city.str.contains(city, case=False),'city'] = city
    
    return dataframe

In [3]:
# unpickle dataset
unpickled_raw_data = pd.read_pickle("./raw_data.pkl")

# manipulate data to become usable
df = manipulate_data(dataframe = unpickled_raw_data)

As we can see the events are being categorized in a consistent way by the police. Therefore I will use the "event" column as a word-vector that summarizes the event.

In [143]:
df.event.value_counts().head(10)

Stöld                586
Trafikolycka         517
Övrigt               292
Rattfylleri          240
Trafikkontroll       212
Brand                209
Trafikbrott          175
Misshandel           100
Fylleri/LOB           69
Arbetsplatsolycka     69
Name: event, dtype: int64

However the nature of "Övrigt" events are quite unambiguous. For that reason, I will try to extract the topic of the event and match it to existed events by analyzing details. In case there is an existed topic, I will change "Övrigt" to the topic, otherwise I will let "Övrigt" as topic.

In [139]:
def word_vectorize():
    # pre-existed events
    events = list(df.event.str.lower().unique())

    # initiate tokenizer
    tokenizer = RegexpTokenizer(r'\w+')

    # swedish stopwords 
    stopwords_list = stopwords.words('swedish')

    # extract nature of event from details (if any) and replace
    for i in df.index:
        if df.loc[i, "event"] == "Övrigt":
            tokenized_text = tokenizer.tokenize(df.loc[i, 'event'])
            for word in tokenized_text:
                if word in stopwords_list:
                    tokenized_text.remove(word)
            for word in tokenized_text:
                if word in events:
                    df.loc[i, 'event'] = word
                break

In [140]:
# run the process
word_vectorize()

## 3. Analyze Data

The data are collected and cleaned up, now I move on to analysis. First, let's take a look on what we have.

### 3.1 Descriptive Statistics

In [4]:
print("- Number of rows: {}".format(df.shape[0]))
print("- Number of columns: {}".format(df.shape[1]))
df.head()

- Number of rows: 2980
- Number of columns: 7


,year,month,day,time,city,event,details
0,2018,11,Sunday,15,Hallands län,Övrigt,Hemsidan kommer inte att uppdateras mer idag. ...
1,2018,11,Sunday,15,Västra götalands län,Övrigt,Hemsidan kommer inte att uppdateras mer idag. ...
2,2018,11,Sunday,15,Botkyrka,Trafikolycka,Trafikolycka på Hallundavägen/ Tomtbergavägen.
3,2018,11,Sunday,13,Malung-sälen,Stöld,Det har varit inbrott i en bostad på Blomsterv...
4,2018,11,Sunday,11,Göteborg,Stöld,I en matvarubutik på Lona Knapes gata i Högsbo...


Let's get an idea of how many different events there are, and how many events per city:

In [5]:
# barplot for frequency of events
events = list(df.event.value_counts().head(20).index)
freq = list(df.event.value_counts().head(20).values)

p = figure(x_range=events, plot_width=900, plot_height=450, title='Frequency of top 20 events')
p.xaxis.major_label_orientation = np.pi/3

p.vbar(x=events, top=freq , width=0.2)
p.xaxis.axis_label = "Events"
p.yaxis.axis_label = "Frequency Of Events"

show(p)

After the manipulation of grouping together the words related to "robbery", we see that the most frequent event is the "Stold". "Trafikolycka" together with "Stold" are the leading events.

When it comes to cities:

In [6]:
# barplot for frequency of events per city
cities = list(df.city.value_counts().head(20).index)
freq = list(df.city.value_counts().head(20).values)

p = figure(x_range=cities, plot_width=900, plot_height=450, title='Top 20 cities with most events')
p.xaxis.major_label_orientation = np.pi/3

p.vbar(x=cities, top=freq , width=0.2)
p.xaxis.axis_label = "City"
p.yaxis.axis_label = "Number Of Events"

show(p)

As it was expected, most of the events take place in Stockholm, while the other cities account for more or less similar number of events. As someone would expect the smaller the city the less number of events occur.

Finally let's take a look at frequency of events based on time of day and day of week:

In [36]:
# heatmap of number of events per day and time
tmp = df[['day', 'time']]
tmp = tmp.groupby(['day', 'time']).size().to_frame('freq').reset_index()

p = HeatMap(tmp, x='time', y='day', values='freq', stat=None, width=900, height=500, 
            title='Heatmap: Frequency per day and time')

show(p)

Since the reports are written by the police officers, it is plausible that the majority of events would gather during working hours. And indeed the heatmap indicates that.

However, as someone would think, e.g. a traffic control would be expected during rush hours, while a more serious event, during night hours. It would be interesting, thus, to analyze how different events cluster based on different features. This is analyzed in the next topic.

### 3.2 Analyzing Nature Of Events With Principal Component Analysis (and t-SNE)

In this section I will cluster events based on cities, day of week, and time of day. In order to get an idea, let's see a dataframe that summarizes the frequency of events, per event and per city.

In [164]:
# create a copy of dataframe
tmp = df.copy()

# only frequent events (more than 20)
most_freq_events = tmp.event.value_counts()
most_freq_events = most_freq_events[most_freq_events.values>20].index
tmp = tmp[tmp.event.isin(most_freq_events)]

most_freq_cities = tmp.city.value_counts()
most_freq_cities = most_freq_cities[most_freq_cities.values>20].index
tmp = tmp[tmp.city.isin(most_freq_cities)]

# manipulate data to get frequency of events for each city
pivot = pd.pivot_table(tmp, aggfunc=len, index=['city'], columns=['event'], fill_value=0, values='details').reset_index()
print("- Number of rows: {}".format(pivot.shape[0]))
print("- Number of columns: {}".format(pivot.shape[1]))
pivot.head()

- Number of rows: 33
- Number of columns: 19


event,city,Arbetsplatsolycka,Brand,Bråk,Fylleri/LOB,Försvunnen person,Kontroll person/fordon,Misshandel,Narkotikabrott,Olaga hot,Rattfylleri,Skadegörelse,Stöld,Trafikbrott,Trafikkontroll,Trafikolycka,Uppdatering,Våld/hot mot tjänsteman,Övrigt
0,Boden,0,3,1,1,0,0,1,0,2,0,0,6,5,16,3,0,1,0
1,Borlänge,2,2,0,0,2,0,2,0,0,2,0,12,0,0,1,0,0,0
2,Eskilstuna,1,4,0,0,1,0,3,1,1,1,0,3,0,0,11,1,0,0
3,Gävle,3,4,2,1,0,0,2,0,0,3,3,17,0,0,3,0,0,0
4,Göteborg,5,5,1,0,0,1,2,1,1,8,1,7,1,0,12,3,0,0


Since I am dealing with a lot of dimensions (33x19), I will use Principal Component Analysis (PCA) in  order to reduce dimensions to 2, and plot the clustering. Of course, given the high dimensionality, I expect some of the information to be lost after the dimensionality reduction. I will also calculate the variance coming from the 2 principal components to estimate the loss of information.

In [211]:
def principal_component_analysis(subject):
    # create a copy of dataframe
    tmp = df.copy()

    # only frequent events (more than 20)
    most_freq_events = tmp.event.value_counts()
    most_freq_events = most_freq_events[most_freq_events.values>20].index
    tmp = tmp[tmp.event.isin(most_freq_events)]

    # only frequent cities (more than 20)
    if subject == 'city':
        most_freq_cities = tmp.city.value_counts()
        most_freq_cities = most_freq_cities[most_freq_cities.values>20].index
        tmp = tmp[tmp.city.isin(most_freq_cities)]

    # manipulate data to get frequency of events for each city
    pivot = pd.pivot_table(tmp, aggfunc=len, index=[subject], columns=['event'], fill_value=0, values='details').reset_index()
    
    # cretae matrix X and vector y as numpy arrays
    X = pivot.ix[:,1:].values
    y = pivot.ix[:,0].values

    # standarize data (mean=0, std=1)
    X_std = StandardScaler().fit_transform(X)

    # initiate pca algorithm and fit data
    pca = PCA(n_components=2)
    pca.fit(X_std)
    pca_data = pca.transform(X_std)

    # build graph
    source = ColumnDataSource(data=dict(principal_component_1=pca_data[:,0],
                                        principal_component_2=pca_data[:,1],
                                        names=pivot[subject].values))

    p = figure(plot_width=900, plot_height=450, title='Principal Component Analysis: event - {}'.format(subject))

    p.scatter(x='principal_component_1', y='principal_component_2', size=10, source=source)
    p.xaxis[0].axis_label = 'Principal Component 1'
    p.yaxis[0].axis_label = 'Principal Component 2'

    labels = LabelSet(x='principal_component_1', y='principal_component_2', text='names', level='glyph', x_offset=5, y_offset=5, source=source, render_mode='canvas')
    p.add_layout(labels)
    
    # variation of the 2 principal components
    per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
    
    variation = Label(x=720, y=380, x_units='screen', y_units='screen',
                 text='Variation = {}'.format(per_var.sum()), render_mode='css',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    
    p.add_layout(variation)
    
    show(p)

In [215]:
principal_component_analysis('city')

The figure is dynamic, one can use the tools to zoom in and out and see the different levels of clustering. The figure shows that cities that are clustered together have similar nature of events.

In [218]:
principal_component_analysis('time')

This figure clusters time of events based on their nature. As we see (and as it was expected) events that happen during the early morning hours have similar nature, one would assume events of more serious nature. On the other hand, on the right side of the figure rush hours are grouped together which was again expected. In the middle section, hours of either early in the morning or right after working hours are grouped.

Finally when it comes to days:

In [217]:
principal_component_analysis('day')

The days are quite spread, therefore, the figure is not very informative.

The elephant in the room, when dealing with PCA is of course the variation. In all the figures, the variation ratio is slightly below or slightly above 50% which means that half of the information of the original high dimensional space was lost during dimensionality reduction. 

For this reason, just for a proof of concept I will run a stochastic dimensionality reduction technique, namely t-SNE for cities, just to get an idea of how geometrical differences are being preserved after dimensionality reduction.

In [84]:
def t_sne(subject):
    # create a copy of dataframe
    tmp = df.copy()

    # only frequent events (more than 20)
    most_freq_events = tmp.event.value_counts()
    most_freq_events = most_freq_events[most_freq_events.values>20].index
    tmp = tmp[tmp.event.isin(most_freq_events)]

    if subject == 'city':
        most_freq_cities = tmp.city.value_counts()
        most_freq_cities = most_freq_cities[most_freq_cities.values>20].index
        tmp = tmp[tmp.city.isin(most_freq_cities)]

    # manipulate data to get frequency of events for each city
    pivot = pd.pivot_table(tmp, aggfunc=len, index=[subject], columns=['event'], fill_value=0, values='details').reset_index()

    # cretae matrix X and vector y as numpy arrays
    X = pivot.ix[:,1:].values
    y = pivot.ix[:,0].values

    # initiate tsne algorithm and fit data
    tsne = TSNE(n_components=2)
    tsne = tsne.fit_transform(X_std)

    # build graph
    source = ColumnDataSource(data=dict(x_axis=tsne[:,0],
                                        y_axis=tsne[:,1],
                                        names=pivot[subject].values))

    p = figure(plot_width=900, plot_height=450, title='t-SNE: event - {}'.format(subject))

    p.scatter(x='x_axis', y='y_axis', size=10, source=source)
    p.xaxis[0].axis_label = 'x'
    p.yaxis[0].axis_label = 'y'

    labels = LabelSet(x='x_axis', y='y_axis', text='names', level='glyph', x_offset=5, y_offset=5, source=source, render_mode='canvas')
    p.add_layout(labels)


    show(p)

In [88]:
t_sne('city')